# HEALTH X - Deep Learning Models

## Contents Of Notebook

1. import statements
2. pulling raw data from data sources
3. defining functions for preprocessing 
4. CNN and CNN + LSTM model for elbow
5. CNN and CNN + LSTM model for hand
6. CNN and CNN + LSTM model for humerus
7. CNN and CNN + LSTM model for forearm
8. CNN and CNN + LSTM model for wrist

### 1. import statements

In [18]:
import pandas as pd
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, LSTM, MaxPooling2D, Conv2D, Flatten, Activation,ConvLSTM2D
path  = 'C:/Users/prave/Documents/Python Work/Big Data AI/Team Project/MURA-v1.1/'
set_random_seed(12567)
seed(109274)

### 2. pulling raw data from data sources

In [5]:
train_paths = pd.read_csv(path+'train_image_paths.csv') # reading training image directories from csv file
train_paths.columns = ['image_paths'] # renaming the column

In [6]:
train_paths.head()

,image_paths
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...


In [8]:
valid_paths = pd.read_csv(path+'valid_image_paths.csv') # reading validation image directories from csv file 
valid_paths.columns = ['image_paths'] # renaming the column

In [9]:
valid_paths.head()

,image_paths
0,MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
1,MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
2,MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
3,MURA-v1.1/valid/XR_WRIST/patient11186/study1_p...
4,MURA-v1.1/valid/XR_WRIST/patient11186/study1_p...


### 3. defining functions for data preprocessing

In [10]:
def prep_dataframe(body_part,t_v): # preparing dataframe for CNN + LSTM
    paths = t_v # pandas DataFrame containing all images
    paths.columns = ['image_paths'] # renaming column
    patient_list = []
    file_list = []
    for i in paths.image_paths: # looping over each file
        if (body_part in i): # get desired body part
            patient_num = i.split('/')[3] # get patient number
            study_num = i.split('/')[4] # get study number
            patient_study = patient_num + '/' + study_num # combining them as patient_number/study_number 
            patient_list.append(patient_study) # appending patient_number/study_number to list
            file_list.append(i) # append file directory to list
    dict_dataset = {'patient':patient_list,'file_name':file_list} # combining the two lists into dictionary
    df_dataset = pd.DataFrame(dict_dataset) # converting to dataframe
    grouped = df_dataset.groupby('patient') # grouping by patient_number/study_number
    final_series = grouped['file_name'].unique() # taking unique values
    final_dataframe = pd.DataFrame({'patient':final_series.index, 'filename_array':final_series.values})
    # getting dataframe with rows in the format: patient_number/study_number  [file1,file2,file3,...]
    final_dataframe['filename_string'] = final_dataframe.filename_array.apply(','.join)
    # converting file names into string which is comma separated 
    file_splits = final_dataframe['filename_string'].str.split(',', expand=True)
    # splitting on comma creates new column for each file name
    patient_files = pd.concat([final_dataframe,file_splits],axis=1)
    # concatenate splits with original dataframe
    patient_files.drop(columns=['filename_array', 'filename_string'],inplace=True)
    # removing unnecessary columns
    return(patient_files)

In [11]:
def img_cnn(df, part, img_size):
    x = []
    y = []
    for i in df.image_paths: # looping over each image
        if (part in i):
            x.append((np.array(cv2.resize(cv2.imread(path+i),(img_size,img_size))))/255)
            # reading image, resizing, normalizing and appending to list
            if 'positive' in i:
                y.append(1) # assigning labels
            else:
                y.append(0)
    return(np.array(x),np.array(y))

In [ ]:
def rnn_df(part):
    train = prep_dataframe(part,train_paths) # preparing dataframe for CNN + LSTM
    train.set_index('patient',inplace=True)
    test = prep_dataframe(part,valid_paths)
    test.set_index('patient',inplace=True)
    if len(train.columns)>len(test.columns):
        l = len(test.columns) 
        # comparing number of columns in training and test data, setting minimum number of columns equal to 
        # the number of columns for both train and test. This is for performance evaluation
        for i in range((len(train.columns)-len(test.columns))):
            test[l+i] = None
    elif len(test.columns)>len(train.columns):
        l = len(train.columns)
        for i in range((len(test.columns))-(len(train.columns))):
            train[l+i] = None   
    return(train,test)

In [15]:
def img_arr(df,img_size):
    zero_pad = [[[0 for k in range(3)] for j in range(img_size)] for i in range(img_size)]
    # creating 2D array of zeroes for missing or None images
    h = []
    for i in df.index:
        c = []
        for j in df.columns:
            if df.loc[i,j]==None:
                c.append(np.array(zero_pad))
            else:
                c.append((np.array(cv2.resize(cv2.imread(path+df.loc[i,j]),(img_size,img_size))))/255)
                # reading image, resizing, normalizing and appending to list
        h.append(np.array(c))
        c.clear()
        
    y = []
    for i in df.index:
        if 'positive' in i: # setting labels
            y.append(1)
        else:
            y.append(0)
    return(np.array(h),np.array(y))
    

In [16]:
def img_rnn(part,img_size):
    train,test = rnn_df(part) # calling rnn_df internally
    x_train, y_train = img_arr(train,img_size) # calling img_arr internally
    x_test, y_test = img_arr(test,img_size) # calling img_arr internally
    return(x_train,y_train,x_test,y_test)    

## 4. ELBOW

### CNN

In [36]:
x_train_elbow,y_train_elbow = img_cnn(train_paths,'ELBOW',64)
x_test_elbow, y_test_elbow = img_cnn(valid_paths,'ELBOW',64)

In [37]:
model_cnn_elbow = Sequential()
model_cnn_elbow.add(Conv2D(64,(3,3), input_shape=[64,64,3]))
model_cnn_elbow.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_elbow.add(Conv2D(32,(3,3)))
model_cnn_elbow.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_elbow.add(Conv2D(16,(3,3)))
model_cnn_elbow.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_elbow.add(Flatten())
model_cnn_elbow.add(Dense(1000))
model_cnn_elbow.add(Activation('relu'))
model_cnn_elbow.add(Dense(1))
model_cnn_elbow.add(Activation('sigmoid'))

model_cnn_elbow.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 29, 29, 32)        18464     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 12, 12, 16)        4624      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 576)               0         
__________

In [38]:
model_cnn_elbow.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [39]:
model_cnn_elbow.fit(x_train_elbow,y_train_elbow,batch_size = 100, epochs = 10)

Epoch 1/10
4931/4931 [==============================] - 31s 6ms/step - loss: 0.6787 - acc: 0.5788
Epoch 2/10
4931/4931 [==============================] - 34s 7ms/step - loss: 0.6517 - acc: 0.6082
Epoch 3/10
4931/4931 [==============================] - 32s 6ms/step - loss: 0.6418 - acc: 0.6299
Epoch 4/10
4931/4931 [==============================] - 31s 6ms/step - loss: 0.6255 - acc: 0.6443
Epoch 5/10
4931/4931 [==============================] - 31s 6ms/step - loss: 0.6088 - acc: 0.6650
Epoch 6/10
4931/4931 [==============================] - 31s 6ms/step - loss: 0.5958 - acc: 0.6773
Epoch 7/10
4931/4931 [==============================] - 31s 6ms/step - loss: 0.5772 - acc: 0.6934
Epoch 8/10
4931/4931 [==============================] - 31s 6ms/step - loss: 0.5463 - acc: 0.7104
Epoch 9/10
4931/4931 [==============================] - 31s 6ms/step - loss: 0.5076 - acc: 0.7487
Epoch 10/10
4931/4931 [==============================] - 31s 6ms/step - loss: 0.4629 - acc: 0.7690


In [40]:
model_cnn_elbow.evaluate(x_test_elbow,y_test_elbow)

465/465 [==============================] - 1s 3ms/step


[0.8057791352272033, 0.6107526888129532]

### CNN + RNN (Time distributed)

In [41]:
x_train_elbow, y_train_elbow, x_test_elbow, y_test_elbow = img_rnn('ELBOW',32)

In [42]:
#CNN
model_mix_elbow = Sequential()
model_mix_elbow.add(TimeDistributed(Conv2D(256,(3,3),activation='relu'),input_shape=(x_train_elbow.shape[1],
                                                                                        x_train_elbow.shape[2],
                                                                                        x_train_elbow.shape[3],
                                                                                        x_train_elbow.shape[4])))
model_mix_elbow.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_elbow.add(TimeDistributed(Conv2D(256,(3,3),activation='relu')))
model_mix_elbow.add(TimeDistributed(Conv2D(128,(3,3),activation='relu')))
model_mix_elbow.add(TimeDistributed(Conv2D(56,(3,3),activation='relu')))
model_mix_elbow.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_elbow.add(TimeDistributed(Conv2D(256,(3,3),activation='relu')))
model_mix_elbow.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_elbow.add(TimeDistributed(Flatten()))

#RNN
model_mix_elbow.add(LSTM(1000,return_sequences=False))

model_mix_elbow.add(Dense(1))
model_mix_elbow.add(Activation('sigmoid'))

model_mix_elbow.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_11 (TimeDis (None, 8, 30, 30, 256)    7168      
_________________________________________________________________
time_distributed_12 (TimeDis (None, 8, 15, 15, 256)    0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 8, 13, 13, 256)    590080    
_________________________________________________________________
time_distributed_14 (TimeDis (None, 8, 11, 11, 128)    295040    
_________________________________________________________________
time_distributed_15 (TimeDis (None, 8, 9, 9, 56)       64568     
_________________________________________________________________
time_distributed_16 (TimeDis (None, 8, 4, 4, 56)       0         
_________________________________________________________________
time_distributed_17 (TimeDis (None, 8, 2, 2, 256)      129280    
__________

In [43]:
model_mix_elbow.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [44]:
model_mix_elbow.fit(x_train_elbow,y_train_elbow,batch_size = 100, epochs = 10)

Epoch 1/10
1754/1754 [==============================] - 231s 132ms/step - loss: 0.6776 - acc: 0.6089
Epoch 2/10
1754/1754 [==============================] - 230s 131ms/step - loss: 0.6670 - acc: 0.6214
Epoch 3/10
1754/1754 [==============================] - 229s 130ms/step - loss: 0.6632 - acc: 0.6237
Epoch 4/10
1754/1754 [==============================] - 228s 130ms/step - loss: 0.6632 - acc: 0.6300
Epoch 5/10
1754/1754 [==============================] - 953s 543ms/step - loss: 0.6519 - acc: 0.6425
Epoch 6/10
1754/1754 [==============================] - 214s 122ms/step - loss: 0.6510 - acc: 0.6363
Epoch 7/10
1754/1754 [==============================] - 213s 121ms/step - loss: 0.6412 - acc: 0.6579
Epoch 8/10
1754/1754 [==============================] - 226s 129ms/step - loss: 0.6523 - acc: 0.6380
Epoch 9/10
1754/1754 [==============================] - 229s 130ms/step - loss: 0.6406 - acc: 0.6505
Epoch 10/10
1754/1754 [==============================] - 229s 131ms/step - loss: 0.6350 - a

In [45]:
model_mix_elbow.evaluate(x_test_elbow,y_test_elbow)

158/158 [==============================] - 6s 36ms/step


[0.5958477368083182, 0.6962025316455697]

-----------------------------------------------------

## 5. HAND

### CNN

In [46]:
x_train_hand,y_train_hand = img_cnn(train_paths,'HAND',32)
x_test_hand,y_test_hand = img_cnn(valid_paths,'HAND',32)

In [47]:
model_cnn_hand = Sequential()
model_cnn_hand.add(Conv2D(64,(3,3), input_shape=[32,32,3]))
model_cnn_hand.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_hand.add(Conv2D(32,(3,3)))
model_cnn_hand.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_hand.add(Conv2D(16,(3,3)))
model_cnn_hand.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_hand.add(Flatten())
model_cnn_hand.add(Dense(1000))
model_cnn_hand.add(Activation('relu'))
model_cnn_hand.add(Dense(1))
model_cnn_hand.add(Activation('sigmoid'))

model_cnn_hand.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 29, 29, 32)        18464     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 12, 12, 16)        4624      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 576)               0         
__________

In [48]:
model_cnn_hand.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [49]:
model_cnn_hand.fit(x_train_hand,y_train_hand,batch_size = 100, epochs = 10)

Epoch 1/10
5543/5543 [==============================] - 36s 6ms/step - loss: 0.5880 - acc: 0.7323
Epoch 2/10
5543/5543 [==============================] - 36s 6ms/step - loss: 0.5760 - acc: 0.7314
Epoch 3/10
5543/5543 [==============================] - 36s 6ms/step - loss: 0.5655 - acc: 0.7328
Epoch 4/10
5543/5543 [==============================] - 36s 6ms/step - loss: 0.5544 - acc: 0.7353
Epoch 5/10
5543/5543 [==============================] - 36s 6ms/step - loss: 0.5400 - acc: 0.7380
Epoch 6/10
5543/5543 [==============================] - 36s 6ms/step - loss: 0.5197 - acc: 0.7453
Epoch 7/10
5543/5543 [==============================] - 36s 6ms/step - loss: 0.4933 - acc: 0.7671
Epoch 8/10
5543/5543 [==============================] - 36s 6ms/step - loss: 0.4673 - acc: 0.7786
Epoch 9/10
5543/5543 [==============================] - 36s 7ms/step - loss: 0.4153 - acc: 0.8089
Epoch 10/10
5543/5543 [==============================] - 36s 7ms/step - loss: 0.3517 - acc: 0.8454


In [50]:
model_cnn_hand.evaluate(x_test_hand,y_test_hand)

460/460 [==============================] - 1s 3ms/step


[0.882223036237385, 0.5695652168730031]

### CNN + RNN (Time Distributed)

In [19]:
x_train_hand, y_train_hand, x_test_hand, y_test_hand = img_rnn('HAND',32)

In [20]:
#CNN
model_mix_hand = Sequential()
model_mix_hand.add(TimeDistributed(Conv2D(256,(3,3),activation='relu'),input_shape=(x_train_hand.shape[1],
                                                                                        x_train_hand.shape[2],
                                                                                        x_train_hand.shape[3],
                                                                                        x_train_hand.shape[4])))
model_mix_hand.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_hand.add(TimeDistributed(Conv2D(256,(3,3),activation='relu')))
model_mix_hand.add(TimeDistributed(Conv2D(128,(3,3),activation='relu')))
model_mix_hand.add(TimeDistributed(Conv2D(56,(3,3),activation='relu')))
model_mix_hand.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_hand.add(TimeDistributed(Conv2D(256,(3,3),activation='relu')))
model_mix_hand.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_hand.add(TimeDistributed(Flatten()))

#RNN
model_mix_hand.add(LSTM(1000,return_sequences=False))

model_mix_hand.add(Dense(1))
model_mix_hand.add(Activation('sigmoid'))

model_mix_hand.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 5, 30, 30, 256)    7168      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 15, 15, 256)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 13, 13, 256)    590080    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 11, 11, 128)    295040    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 5, 9, 9, 56)       64568     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 5, 4, 4, 56)       0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 5, 2, 2, 256)      129280    
__________

In [21]:
model_mix_hand.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [22]:
model_mix_hand.fit(x_train_hand,y_train_hand,batch_size = 100, epochs = 10)

Epoch 1/10
2018/2018 [==============================] - 225s 111ms/step - loss: 0.6251 - acc: 0.7418
Epoch 2/10
2018/2018 [==============================] - 214s 106ms/step - loss: 0.5764 - acc: 0.7418
Epoch 3/10
2018/2018 [==============================] - 217s 108ms/step - loss: 0.5724 - acc: 0.7418
Epoch 4/10
2018/2018 [==============================] - 217s 108ms/step - loss: 0.5658 - acc: 0.7418
Epoch 5/10
2018/2018 [==============================] - 214s 106ms/step - loss: 0.5644 - acc: 0.7418
Epoch 6/10
2018/2018 [==============================] - 211s 105ms/step - loss: 0.5599 - acc: 0.7418
Epoch 7/10
2018/2018 [==============================] - 213s 106ms/step - loss: 0.5542 - acc: 0.7418
Epoch 8/10
2018/2018 [==============================] - 213s 105ms/step - loss: 0.5573 - acc: 0.7433
Epoch 9/10
2018/2018 [==============================] - 220s 109ms/step - loss: 0.5546 - acc: 0.7423
Epoch 10/10
2018/2018 [==============================] - 221s 109ms/step - loss: 0.5490 - a

In [23]:
model_mix_hand.evaluate(x_test_hand,y_test_hand)

167/167 [==============================] - 5s 33ms/step


[0.7639048422525029, 0.6047904191616766]

## 6. HUMERUS

### CNN

In [56]:
x_train_humerus,y_train_humerus = img_cnn(train_paths,'HUMERUS',64)
x_test_humerus,y_test_humerus = img_cnn(valid_paths,'HUMERUS',64)

In [57]:
model_cnn_humerus = Sequential()
model_cnn_humerus.add(Conv2D(64,(3,3), input_shape=[64,64,3]))
model_cnn_humerus.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_humerus.add(Conv2D(32,(3,3)))
model_cnn_humerus.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_humerus.add(Conv2D(16,(3,3)))
model_cnn_humerus.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_humerus.add(Flatten())
model_cnn_humerus.add(Dense(1000))
model_cnn_humerus.add(Activation('relu'))
model_cnn_humerus.add(Dense(1))
model_cnn_humerus.add(Activation('sigmoid'))

model_cnn_humerus.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 29, 29, 32)        18464     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 12, 12, 16)        4624      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 576)               0         
__________

In [58]:
model_cnn_humerus.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [59]:
model_cnn_humerus.fit(x_train_humerus,y_train_humerus,batch_size = 100, epochs = 10)

Epoch 1/10
1272/1272 [==============================] - 9s 7ms/step - loss: 0.6967 - acc: 0.5244
Epoch 2/10
1272/1272 [==============================] - 8s 6ms/step - loss: 0.6712 - acc: 0.5983
Epoch 3/10
1272/1272 [==============================] - 8s 7ms/step - loss: 0.6438 - acc: 0.6305
Epoch 4/10
1272/1272 [==============================] - 8s 7ms/step - loss: 0.6372 - acc: 0.6439
Epoch 5/10
1272/1272 [==============================] - 8s 7ms/step - loss: 0.6137 - acc: 0.6478
Epoch 6/10
1272/1272 [==============================] - 9s 7ms/step - loss: 0.5877 - acc: 0.6903
Epoch 7/10
1272/1272 [==============================] - 8s 7ms/step - loss: 0.5378 - acc: 0.7296
Epoch 8/10
1272/1272 [==============================] - 9s 7ms/step - loss: 0.4985 - acc: 0.7547
Epoch 9/10
1272/1272 [==============================] - 9s 7ms/step - loss: 0.4638 - acc: 0.7618
Epoch 10/10
1272/1272 [==============================] - 9s 7ms/step - loss: 0.4618 - acc: 0.7783


In [60]:
model_cnn_humerus.evaluate(x_test_humerus,y_test_humerus)

288/288 [==============================] - 1s 4ms/step


[0.6979264186488258, 0.6423611111111112]

### CNN + RNN (Time Distributed)

In [61]:
x_train_humerus, y_train_humerus, x_test_humerus, y_test_humerus = img_rnn('HUMERUS',32)

In [62]:
#CNN
model_mix_humerus = Sequential()
model_mix_humerus.add(TimeDistributed(Conv2D(64,(3,3),activation='relu'),input_shape=(x_train_humerus.shape[1],
                                                                                      x_train_humerus.shape[2],
                                                                                      x_train_humerus.shape[3],
                                                                                      x_train_humerus.shape[4])))
model_mix_humerus.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_humerus.add(TimeDistributed(Conv2D(32,(3,3),activation='relu')))
model_mix_humerus.add(TimeDistributed(Conv2D(128,(3,3),activation='relu')))
model_mix_humerus.add(TimeDistributed(Conv2D(56,(3,3),activation='relu')))
model_mix_humerus.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_humerus.add(TimeDistributed(Conv2D(256,(3,3),activation='relu')))
model_mix_humerus.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))


model_mix_humerus.add(TimeDistributed(Flatten()))

#RNN
model_mix_humerus.add(LSTM(100,return_sequences=False))

model_mix_humerus.add(Dense(1))
model_mix_humerus.add(Activation('sigmoid'))

model_mix_humerus.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_29 (TimeDis (None, 6, 30, 30, 64)     1792      
_________________________________________________________________
time_distributed_30 (TimeDis (None, 6, 15, 15, 64)     0         
_________________________________________________________________
time_distributed_31 (TimeDis (None, 6, 13, 13, 32)     18464     
_________________________________________________________________
time_distributed_32 (TimeDis (None, 6, 6, 6, 32)       0         
_________________________________________________________________
time_distributed_33 (TimeDis (None, 6, 1152)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               501200    
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 101       
__________

In [63]:
model_mix_humerus.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [64]:
model_mix_humerus.fit(x_train_humerus,y_train_humerus,batch_size = 100, epochs = 10)

Epoch 1/10
592/592 [==============================] - 7s 12ms/step - loss: 0.6946 - acc: 0.5574
Epoch 2/10
592/592 [==============================] - 6s 9ms/step - loss: 0.6898 - acc: 0.5642
Epoch 3/10
592/592 [==============================] - 6s 10ms/step - loss: 0.6874 - acc: 0.5845
Epoch 4/10
592/592 [==============================] - 6s 10ms/step - loss: 0.6817 - acc: 0.5608
Epoch 5/10
592/592 [==============================] - 6s 9ms/step - loss: 0.6714 - acc: 0.5878
Epoch 6/10
592/592 [==============================] - 6s 10ms/step - loss: 0.6571 - acc: 0.6115
Epoch 7/10
592/592 [==============================] - 6s 10ms/step - loss: 0.6541 - acc: 0.6385
Epoch 8/10
592/592 [==============================] - 5s 9ms/step - loss: 0.6354 - acc: 0.6402
Epoch 9/10
592/592 [==============================] - 6s 10ms/step - loss: 0.6006 - acc: 0.6824
Epoch 10/10
592/592 [==============================] - 5s 9ms/step - loss: 0.5783 - acc: 0.6807


In [65]:
model_mix_humerus.evaluate(x_test_humerus,y_test_humerus)

135/135 [==============================] - 1s 7ms/step


[0.6975569367408753, 0.548148148589664]

## 7. Forearm

### CNN

In [66]:
x_train_forearm,y_train_forearm = img_cnn(train_paths,'FOREARM',64)
x_test_forearm,y_test_forearm = img_cnn(valid_paths,'FOREARM',64)

In [67]:
model_cnn_forearm = Sequential()
model_cnn_forearm.add(Conv2D(64,(3,3), input_shape=[64,64,3]))
model_cnn_forearm.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_forearm.add(Conv2D(32,(3,3)))
model_cnn_forearm.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_forearm.add(Conv2D(16,(3,3)))
model_cnn_forearm.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_forearm.add(Flatten())
model_cnn_forearm.add(Dense(1000))
model_cnn_forearm.add(Activation('relu'))
model_cnn_forearm.add(Dense(1))
model_cnn_forearm.add(Activation('sigmoid'))

model_cnn_forearm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 29, 29, 32)        18464     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 12, 12, 16)        4624      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 576)               0         
__________

In [68]:
model_cnn_forearm.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [69]:
model_cnn_forearm.fit(x_train_forearm,y_train_forearm,batch_size = 100, epochs = 10)

Epoch 1/10
1825/1825 [==============================] - 13s 7ms/step - loss: 0.6609 - acc: 0.6290
Epoch 2/10
1825/1825 [==============================] - 13s 7ms/step - loss: 0.6409 - acc: 0.6438
Epoch 3/10
1825/1825 [==============================] - 12s 7ms/step - loss: 0.6298 - acc: 0.6542
Epoch 4/10
1825/1825 [==============================] - 13s 7ms/step - loss: 0.6053 - acc: 0.6816
Epoch 5/10
1825/1825 [==============================] - 12s 7ms/step - loss: 0.5786 - acc: 0.7123
Epoch 6/10
1825/1825 [==============================] - 12s 7ms/step - loss: 0.5623 - acc: 0.7118
Epoch 7/10
1825/1825 [==============================] - 12s 7ms/step - loss: 0.5508 - acc: 0.7282
Epoch 8/10
1825/1825 [==============================] - 13s 7ms/step - loss: 0.5197 - acc: 0.7441
Epoch 9/10
1825/1825 [==============================] - 14s 8ms/step - loss: 0.5187 - acc: 0.7501
Epoch 10/10
1825/1825 [==============================] - 12s 7ms/step - loss: 0.4907 - acc: 0.7710


In [70]:
model_cnn_forearm.evaluate(x_test_forearm,y_test_forearm)

301/301 [==============================] - 1s 4ms/step


[0.6905128825146495, 0.6245847182020396]

### CNN + RNN (Time Distributed)

In [42]:
x_train_forearm, y_train_forearm, x_test_forearm, y_test_forearm = img_rnn('FOREARM',32)

In [43]:
#CNN
model_mix_forearm = Sequential()
model_mix_forearm.add(TimeDistributed(Conv2D(256,(3,3),activation='relu'),input_shape=(x_train_forearm.shape[1],
                                                                                       x_train_forearm.shape[2],
                                                                                       x_train_forearm.shape[3],
                                                                                       x_train_forearm.shape[4])))
model_mix_forearm.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_forearm.add(TimeDistributed(Conv2D(256,(3,3),activation='relu')))
model_mix_forearm.add(TimeDistributed(Conv2D(128,(3,3),activation='relu')))
model_mix_forearm.add(TimeDistributed(Conv2D(56,(3,3),activation='relu')))
model_mix_forearm.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

#model_mix_forearm.add(TimeDistributed(Conv2D(256,(3,3),activation='relu')))
#model_mix_forearm.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_forearm.add(TimeDistributed(Flatten()))

#RNN
model_mix_forearm.add(LSTM(1000,return_sequences=False))

model_mix_forearm.add(Dense(1))
model_mix_forearm.add(Activation('sigmoid'))

model_mix_forearm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_22 (TimeDis (None, 10, 30, 30, 256)   7168      
_________________________________________________________________
time_distributed_23 (TimeDis (None, 10, 15, 15, 256)   0         
_________________________________________________________________
time_distributed_24 (TimeDis (None, 10, 13, 13, 256)   590080    
_________________________________________________________________
time_distributed_25 (TimeDis (None, 10, 11, 11, 128)   295040    
_________________________________________________________________
time_distributed_26 (TimeDis (None, 10, 9, 9, 56)      64568     
_________________________________________________________________
time_distributed_27 (TimeDis (None, 10, 4, 4, 56)      0         
_________________________________________________________________
time_distributed_28 (TimeDis (None, 10, 896)           0         
__________

In [44]:
model_mix_forearm.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [46]:
model_mix_forearm.fit(x_train_forearm,y_train_forearm,batch_size = 100, epochs = 10)

Epoch 1/10
877/877 [==============================] - 192s 219ms/step - loss: 0.6601 - acc: 0.6727
Epoch 2/10
877/877 [==============================] - 186s 212ms/step - loss: 0.6599 - acc: 0.6727
Epoch 3/10
877/877 [==============================] - 257s 293ms/step - loss: 0.6396 - acc: 0.6727
Epoch 4/10
877/877 [==============================] - 189s 216ms/step - loss: 0.6344 - acc: 0.6727
Epoch 5/10
877/877 [==============================] - 184s 210ms/step - loss: 0.6330 - acc: 0.6727
Epoch 6/10
877/877 [==============================] - 185s 211ms/step - loss: 0.6311 - acc: 0.6727
Epoch 7/10
877/877 [==============================] - 186s 212ms/step - loss: 0.6318 - acc: 0.6705
Epoch 8/10
877/877 [==============================] - 186s 213ms/step - loss: 0.6184 - acc: 0.6773
Epoch 9/10
877/877 [==============================] - 4876s 6s/step - loss: 0.6091 - acc: 0.6910
Epoch 10/10
877/877 [==============================] - 189s 215ms/step - loss: 0.6109 - acc: 0.6944


In [48]:
model_mix_forearm.evaluate(x_test_forearm,y_test_forearm)

133/133 [==============================] - 7s 54ms/step


[0.7247540229245236, 0.5263157894736842]

## 8. WRIST

### CNN

In [40]:
x_train_wrist,y_train_wrist = img_cnn(train_paths,'WRIST',32)
x_test_wrist,y_test_wrist = img_cnn(valid_paths,'WRIST',32)

In [41]:
model_cnn_wrist = Sequential()
model_cnn_wrist.add(Conv2D(64,(3,3), input_shape=[32,32,3]))
model_cnn_wrist.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_wrist.add(Conv2D(32,(3,3)))
model_cnn_wrist.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_wrist.add(Conv2D(16,(3,3)))
model_cnn_wrist.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_wrist.add(Flatten())
model_cnn_wrist.add(Dense(1000))
model_cnn_wrist.add(Activation('relu'))
model_cnn_wrist.add(Dense(1))
model_cnn_wrist.add(Activation('sigmoid'))

model_cnn_wrist.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 13, 13, 32)        18464     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 4, 4, 16)          4624      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 2, 2, 16)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 64)                0         
__________

In [42]:
model_cnn_wrist.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [43]:
model_cnn_wrist.fit(x_train_wrist,y_train_wrist,batch_size = 100, epochs = 10)

Epoch 1/10
9752/9752 [==============================] - 14s 1ms/step - loss: 0.6514 - acc: 0.6111
Epoch 2/10
9752/9752 [==============================] - 13s 1ms/step - loss: 0.6157 - acc: 0.6588
Epoch 3/10
9752/9752 [==============================] - 14s 1ms/step - loss: 0.5821 - acc: 0.6937
Epoch 4/10
9752/9752 [==============================] - 14s 1ms/step - loss: 0.5527 - acc: 0.7242
Epoch 5/10
9752/9752 [==============================] - 15s 2ms/step - loss: 0.5231 - acc: 0.7395
Epoch 6/10
9752/9752 [==============================] - 15s 2ms/step - loss: 0.5017 - acc: 0.7582
Epoch 7/10
9752/9752 [==============================] - 15s 2ms/step - loss: 0.4918 - acc: 0.7594
Epoch 8/10
9752/9752 [==============================] - 15s 2ms/step - loss: 0.4670 - acc: 0.7759
Epoch 9/10
9752/9752 [==============================] - 15s 2ms/step - loss: 0.4467 - acc: 0.7937
Epoch 10/10
9752/9752 [==============================] - 15s 2ms/step - loss: 0.4362 - acc: 0.7990


In [44]:
model_cnn_wrist.evaluate(x_test_wrist,y_test_wrist)

658/658 [==============================] - 1s 1ms/step


[0.6180097511474122, 0.7006079034602388]

### CNN + RNN (Time Distributed)

In [30]:
x_train_wrist, y_train_wrist, x_test_wrist, y_test_wrist = img_rnn('WRIST',32)

In [34]:
#CNN
model_mix_wrist = Sequential()
model_mix_wrist.add(TimeDistributed(Conv2D(256,(3,3),activation='relu'),input_shape=(x_train_wrist.shape[1],
                                                                                        x_train_wrist.shape[2],
                                                                                        x_train_wrist.shape[3],
                                                                                        x_train_wrist.shape[4])))
model_mix_wrist.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_wrist.add(TimeDistributed(Conv2D(256,(3,3),activation='relu')))
model_mix_wrist.add(TimeDistributed(Conv2D(128,(3,3),activation='relu')))
model_mix_wrist.add(TimeDistributed(Conv2D(56,(3,3),activation='relu')))
model_mix_wrist.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_wrist.add(TimeDistributed(Conv2D(256,(3,3),activation='relu')))
model_mix_wrist.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_wrist.add(TimeDistributed(Flatten()))

#RNN
model_mix_wrist.add(LSTM(1000,return_sequences=False))

model_mix_wrist.add(Dense(1))
model_mix_wrist.add(Activation('sigmoid'))

model_mix_wrist.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, 8, 30, 30, 256)    7168      
_________________________________________________________________
time_distributed_20 (TimeDis (None, 8, 15, 15, 256)    0         
_________________________________________________________________
time_distributed_21 (TimeDis (None, 8, 13, 13, 256)    590080    
_________________________________________________________________
time_distributed_22 (TimeDis (None, 8, 11, 11, 128)    295040    
_________________________________________________________________
time_distributed_23 (TimeDis (None, 8, 9, 9, 56)       64568     
_________________________________________________________________
time_distributed_24 (TimeDis (None, 8, 4, 4, 56)       0         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 8, 2, 2, 256)      129280    
__________

In [36]:
model_mix_wrist.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [37]:
model_mix_wrist.fit(x_train_wrist,y_train_wrist,batch_size = 100, epochs = 10)

Epoch 1/10
3460/3460 [==============================] - 591s 171ms/step - loss: 0.6726 - acc: 0.6104
Epoch 2/10
3460/3460 [==============================] - 451s 130ms/step - loss: 0.6600 - acc: 0.6107
Epoch 3/10
3460/3460 [==============================] - 543s 157ms/step - loss: 0.6420 - acc: 0.6231
Epoch 4/10
3460/3460 [==============================] - 547s 158ms/step - loss: 0.6187 - acc: 0.6402
Epoch 5/10
3460/3460 [==============================] - 680s 197ms/step - loss: 0.6062 - acc: 0.6659
Epoch 6/10
3460/3460 [==============================] - 598s 173ms/step - loss: 0.5921 - acc: 0.6679
Epoch 7/10
3460/3460 [==============================] - 527s 152ms/step - loss: 0.5869 - acc: 0.6737
Epoch 8/10
3460/3460 [==============================] - 588s 170ms/step - loss: 0.5615 - acc: 0.7139
Epoch 9/10
3460/3460 [==============================] - 571s 165ms/step - loss: 0.5302 - acc: 0.7382
Epoch 10/10
3460/3460 [==============================] - 511s 148ms/step - loss: 0.5638 - a

In [39]:
model_mix_wrist.evaluate(x_test_wrist,y_test_wrist)

237/237 [==============================] - 8s 33ms/step


[0.5752480022514923, 0.6962025329030516]